In [ ]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cmocean as cmo
import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy as np
import copy
import sys,os

sys.path.append('../src')

from rundata import RunData
from alldata import AllData

np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Input parameters

In [ ]:
newrun  = 'tot2'  # New run to be compared to control
depthT = 370
depthS = 100

Get 2d variables

In [ ]:
ad = AllData()

rd1 = RunData(newrun)
rd1.checkfornewdata()
rd1.get_var2d('thetao',depthT)
rd1.get_var2d('so',depthS)

rd0 = RunData('ctrl')
rd0.get_var2d('thetao',depthT,rd1.nmonths)
rd0.get_var2d('so',depthS,rd1.nmonths)

Get basin mask for inset

In [ ]:
ds = xr.open_dataset(f'../data/basinmask.nc').isel(y=slice(0,rd1.jmax),basin=slice(0,5))
mask = np.nansum(ds.mask,axis=1)
ds.close()

Get monthly teperature time series

In [ ]:
ds = xr.open_dataset(f'../data/temperature_mon_ctrl.nc').isel(basin=slice(0,5))
tempc = ds.temp.isel(time=slice(0,rd1.nmonths))
ds.close()

ds = xr.open_dataset(f'../data/temperature_mon_{newrun}.nc').isel(basin=slice(0,5))
time = ds.time.values/12
tempr = ds.temp
ds.close()
dtemp = tempr-tempc

In [ ]:
cmaps = {}
vmins = {}
vmaxs = {}

cmaps['thetao'] = 'cmo.balance'
cmaps['so']     = 'cmo.tarn_r'
vmins['thetao'] = -2
vmins['so']     = -1
vmaxs['thetao'] = 2
vmaxs['so']     = 1

varname = {}
varname['thetao'] = 'Temperature'
varname['so']     = 'Salinity'

mpl.rc('figure',figsize=(15,5))
mpl.rc('figure.subplot',wspace=.15,hspace=.01,left=.01,right=.99,bottom=.1,top=.9)

mnames = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [ ]:
#Save frames

prefix = f'substemp_{newrun}_mon'

framerate = rd1.nmonths/60

#for m in range(nmonths-2,nmonths):
for m in range(rd1.nmonths):
    fig = plt.figure()
    
    for v,vv in enumerate(['so','thetao']):
        ax = fig.add_subplot(1,3,v+1,projection=ccrs.SouthPolarStereo())
        ax.set_extent([-180, 180, -80, -59], crs=ccrs.PlateCarree())
        im = ax.scatter(rd1.lon,rd1.lat,5,c=rd1.var2d[vv][m,:,:]-rd0.var2d[vv][m,:,:],cmap=cmaps[vv],vmin=vmins[vv],vmax=vmaxs[vv],transform=ccrs.PlateCarree())
        ax.add_feature(cartopy.feature.LAND.with_scale('10m'),facecolor='.7')
        ax.coastlines(linewidth=0.4, zorder=2, resolution='10m')
        axins = inset_axes(ax,width="5%",height="100%",loc='center right',borderpad=-2)
        cb = fig.colorbar(im, cax=axins)
    
        ax.set_title(f'{varname[vv]} anomaly at {rd1.dep2d[vv]:.0f}m depth [degC]')
        ax.set_axis_off()

    ax = fig.add_subplot(133)
    ax.axhline(0,0,1,color='.5',ls=':')
    for b,bas in enumerate(ad.basin):
        ax.plot(time[:m+1],dtemp.sel(basin=bas)[:m+1],c=ad.bcol[bas])
        ax.scatter(time[m],dtemp.sel(basin=bas)[m],40,c=ad.bcol[bas])
    ax.set_xlim([0,np.ceil(rd1.nmonths/12)])
    ax.set_ylim([vmins['thetao'],vmaxs['thetao']]) 
    ax.set_yticklabels([])
    ax.set_title('Subsurface ocean temperature anomaly [degC]')
    ax.set_xlabel('Years')

    axins = inset_axes(ax,width="30%",height="30%",loc='lower left',borderpad=0,axes_class=cartopy.mpl.geoaxes.GeoAxes,axes_kwargs=dict(map_projection=cartopy.crs.SouthPolarStereo()))
    axins.set_extent([-180, 180, -80, -60], crs=ccrs.PlateCarree())
    for b,bas in enumerate(ad.basin):
        axins.scatter(rd1.lon,rd1.lat,5*mask[b,:,:],c=ad.bcol[bas],lw=0,transform=ccrs.PlateCarree())

    axins.add_feature(cartopy.feature.LAND.with_scale('10m'),facecolor='.7')
    axins.coastlines(linewidth=0.4, zorder=2, resolution='10m')
    axins.set_axis_off()
    
    y2d = int(np.floor(m/12))
    m2d = int(m-12*y2d)
    fig.suptitle(f'{mnames[m2d]} {y2d:03.0f} of {rd1.nyears:03.0f}')
    
    savename = f'../videos/{prefix}_{m:04.0f}.png'
    plt.savefig(savename,dpi=150,facecolor='w',transparent=False)
    print('Saved',m+1,'of',rd1.nmonths,'months',end='\r')
    plt.close()
    

In [ ]:
#Make video and delete frames
os.system(f'ffmpeg -y -r {framerate} -f image2 -s 1920x1080 -i ../videos/{prefix}_%04d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p ../videos/{prefix}.mp4')
os.system(f'rm -r ../videos/{prefix}*.png')